# 導入函式庫

In [54]:
import pandas_ta as ta
import pandas as pd
import ccxt
import plotly as plt
import plotly.graph_objects as go
import vectorbt as vb
import numpy
import warnings

# 取得交易所數據(利用ccxt)

In [ ]:
exchange=ccxt.binance({
    "api_key":"kVxc7iv6jJaTiXG4iPlVSOHvESaFs8mndYwW8vyf9ygBu7XaatkIVsbpBtOpUuRD",
    "secret":"ophoMQWGuGCbT8RYMm6R1SvsYyC1zlBHgxBBiVVjEvkbtSNHnuoRCOSjNj4urKO0",
    "enableRateLimit":True,
})
exchange.load_markets()

In [56]:
symbol="BTC/USDT"
timeframe="1d"

candles=exchange.fetch_ohlcv(symbol,timeframe=timeframe,limit=None)
df=pd.DataFrame(candles[:],columns=["timestamp","open","high","low","close","volume"])
df["dt"]=pd.to_datetime(df["timestamp"],unit="ms")

# 印出dataframe

In [57]:
df

,timestamp,open,high,low,close,volume,dt
0,1608768000000,23232.39,23794.43,22703.42,23729.20,69013.834252,2020-12-24
1,1608854400000,23728.99,24789.86,23433.60,24712.47,79519.943569,2020-12-25
2,1608940800000,24712.47,26867.03,24500.00,26493.39,97806.513386,2020-12-26
3,1609027200000,26493.40,28422.00,25700.00,26281.66,148455.586214,2020-12-27
4,1609113600000,26281.54,27500.00,26101.00,27079.41,79721.742496,2020-12-28
...,...,...,...,...,...,...,...
495,1651536000000,38525.16,38651.51,37517.80,37728.95,40316.453580,2022-05-03
496,1651622400000,37728.95,40023.77,37670.00,39690.00,62574.617360,2022-05-04
497,1651708800000,39690.00,39845.51,35571.90,36552.97,88722.433550,2022-05-05
498,1651795200000,36552.97,36675.63,35258.00,36013.77,68437.801870,2022-05-06


# 將ema計算好，放入dataframe中，方便之後使用

$$
EMA_{today}=(Value_{today}*(\frac{smoothing}{1+Days}))+EMA_{yesterday}*(1-(\frac{smoothing}{1+Days})) \\[0.3cm]
$$

In [58]:
df['ema']=round(ta.ema(df['close'],length=200),2)

In [59]:
entries=df.apply(lambda x:(x['close']<x['ema']),axis=1)
exits=df.apply(lambda x:(x['close']>x['ema']),axis=1)
pf=vb.Portfolio.from_signals(df['close'],entries=entries,exits=exits,init_cash=10000,fees=0.001)

In [62]:
warnings.simplefilter(action='ignore',category=FutureWarning)
pf.stats()

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                             0.000000
End                             499.000000
Period                          500.000000
Start Value                   10000.000000
End Value                     11582.716559
Total Return [%]                 15.827166
Benchmark Return [%]             51.252760
Max Gross Exposure [%]          100.000000
Total Fees Paid                 178.328926
Max Drawdown [%]                 30.896670
Max Drawdown Duration           132.000000
Total Trades                      7.000000
Total Closed Trades               6.000000
Total Open Trades                 1.000000
Open Trade PnL                -2363.129833
Win Rate [%]                     83.333333
Best Trade [%]                   24.813295
Worst Trade [%]                  -7.832537
Avg Winning Trade [%]             8.984290
Avg Losing Trade [%]             -7.832537
Avg Winning Trade Duration       11.200000
Avg Losing Trade Duration        90.000000
Profit Factor                     4.333047
Expectancy 

In [63]:
fig=go.Figure(data=[go.Candlestick( x=df['dt'],
                                    open=df['open'],
                                    high=df['high'],
                                    low=df['low'],
                                    close=df['close'],
                                    name="k棒"),
                                    
                    go.Scatter     (x=df['dt'],
                                    y=df['ema'],
                                    line=dict(color='black',width=2),
                                    name="200 ema")
                    ]
)

fig.update_layout(  xaxis_rangeslider_visible=False,
                    title="BTC 1DAY with ema",)
fig.show()